In [ ]:
!wget http://eddylab.org/software/hmmer/hmmer-3.4.tar.gz

In [2]:
!tar -xzf hmmer-3.4.tar.gz
!rm hmmer-3.4.tar.gz

tar (child): hmmer-3.4.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
rm: cannot remove 'hmmer-3.4.tar.gz': No such file or directory


In [ ]:
%cd hmmer-3.4
!./configure
!make

In [ ]:
!mkdir ../../dataset/HMM

In [ ]:
%cd ../../dataset/HMM
!wget https://ftp.ebi.ac.uk/pub/databases/Pfam/current_release/Pfam-A.hmm.gz

In [ ]:
!gunzip Pfam-A.hmm.gz

In [ ]:
%cd ../../experiments/HMM

In [ ]:
!hmmer-3.4/src/hmmpress ../../dataset/HMM/Pfam-A.hmm

In [ ]:
!hmmer-3.4/src/hmmscan --cpu 24 --domtblout ../../dataset/HMM/test_pfam_results.domtblout ../../dataset/HMM/Pfam-A.hmm ../../dataset/ec40/test.fasta

In [ ]:
!hmmer-3.4/src/hmmscan --cpu 24 --domtblout ../../dataset/HMM/valid_pfam_results.domtblout ../../dataset/HMM/Pfam-A.hmm ../../dataset/ec40/valid.fasta

In [ ]:
!hmmer-3.4/src/hmmscan --cpu 24 --domtblout ../../dataset/HMM/train_pfam_results.domtblout ../../dataset/HMM/Pfam-A.hmm ../../dataset/ec40/train.fasta

In [22]:
import pandas as pd

def parse_HMM(flag="train"):
    file_path = f"../../dataset/HMM/{flag}_pfam_results.domtblout"
    output_path = f"../../dataset/HMM/{flag}_features.csv"
    # Define column names based on HMMER domtblout format
    columns = [
        "target_name", "accession", "tlen", "query_name", "query_accession", "qlen","E-value", "score", "bias", "#", "of", "c-Evalue", "i-Evalue", "domain_score","domain_bias", "hmm_from", "hmm_to", "ali_from", "ali_to", "env_from", "env_to","acc", "description"
    ]

    # Read the domtblout file, ignoring comment lines (#)
    data = []
    with open(file_path, "r") as f:
        for line in f:
            if line.startswith("#"):  # Ignore comment lines
                continue
            cols = line.strip().split(maxsplit=22)  # Split into 23 columns (last one is description)
            data.append(cols)

    # Convert to DataFrame
    df = pd.DataFrame(data, columns=columns)
    df[['ali_to', 'ali_from', 'qlen']] = df[['ali_to', 'ali_from', 'qlen']].apply(pd.to_numeric, errors='coerce')

   # Compute coverage
    df['coverage'] = (df['ali_to'] - df['ali_from'] + 1) / df['qlen']

    # Group by each query to aggregate
    agg_df = df.groupby("query_name").agg({
        "E-value": "min",          # best (lowest) E-value
        "score": "max",            # best (highest) bit score
        "coverage": "sum",         # total coverage
        "target_name": "count"     # number of domains
    }).reset_index().rename(columns={"target_name": "num_domains"})

    # # One-hot encode domain presence
    # domain_onehot = pd.get_dummies(df[["query_name", "target_name"]].drop_duplicates(), columns=["target_name"])
    # domain_onehot = domain_onehot.groupby("query_name").sum().reset_index()
    # final_features = pd.merge(agg_df, domain_onehot, on="query_name", how="left").fillna(0)
    agg_df.to_csv(output_path, index=False)

In [23]:
parse_HMM("train")
parse_HMM("test")
parse_HMM("valid")

In [24]:
pd.read_csv("../../dataset/HMM/test_features.csv")

,query_name,E-value,score,coverage,num_domains
0,A0A060X6Z0,1.300000e-33,572.2,0.918200,5
1,A0A0A1GKA2,3.500000e-04,64.0,3.365657,27
2,A0A0A1GNF2,3.400000e-64,217.0,0.716724,1
3,A0A0B6VIJ5,1.100000e-01,91.2,0.754717,3
4,A0A0J9X1Q5,2.900000e-02,9.8,0.878077,3
...,...,...,...,...,...
5445,Q9ZYM7,3.700000e-03,30.3,1.434783,17
5446,Q9ZZX0,3.400000e-02,53.1,0.985876,8
5447,R4J7Z9,3.400000e-02,326.9,1.147500,9
5448,W0HJ53,3.700000e-81,271.4,0.824074,1
